In [1]:
import logging

import attacks
import datasets
import evaluate
from intrusion_detection_systems import apollon, metrics, models
from intrusion_detection_systems.apollon import MABThompsonSampling, MABEpsilonGreedy

#
# Evaluate models and MABs against specified attack on specified dataset
#


#####################
# Set up evaluation #
#####################

logging.basicConfig(level=logging.INFO)

seed = 42
n = 500

dataset = datasets.CIC2017SubSample
attack = attacks.ZOO

%pwd

/opt/homebrew/Caskroom/miniconda/base/envs/apollon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniconda/base/envs/apollon/lib/python3.11/site-packages/art/estimators/certification/__init__.py:29: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


'/Users/lucas/Documents/06 Tech/Programming/Python/apollon/main'

In [2]:
# Change to root directory
%cd ..

/Users/lucas/Documents/06 Tech/Programming/Python/apollon


In [3]:
dataset = dataset(load_preprocessed_data=False, seed=seed)

labels: {'Web Attack � Sql Injection', 'PortScan', 'FTP-Patator', 'SSH-Patator', 'Bot', 'Web Attack � XSS', 'Infiltration', 'DDoS', 'Web Attack � Brute Force'}


# Fit models

In [4]:
mlp = models.MLP(random_state=seed)
mlp.fit(dataset.X_train, dataset.y_train)

MLPClassifier(activation='tanh', batch_size=200, early_stopping=True,
              hidden_layer_sizes=(32,), random_state=42)

In [5]:
rf = models.RF(random_state=seed)
rf.fit(dataset.X_train, dataset.y_train)

RandomForestClassifier(random_state=42)

In [6]:
dt = models.DT(random_state=seed)
dt.fit(dataset.X_train, dataset.y_train)

DecisionTreeClassifier(random_state=42)

In [7]:
nb = models.NB()
nb.fit(dataset.X_train, dataset.y_train)

GaussianNB()

In [8]:
lr = models.LR(random_state=seed)
lr.fit(dataset.X_train, dataset.y_train)

/opt/homebrew/Caskroom/miniconda/base/envs/apollon/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [9]:
gbc = models.GBC(random_state=seed)
gbc.fit(dataset.X_train, dataset.y_train)

GradientBoostingClassifier(random_state=42)

In [10]:
et = models.ET(random_state=seed)
et.fit(dataset.X_train, dataset.y_train)

ExtraTreesClassifier(random_state=42)

In [11]:
from typing import Optional, Union, List
from art.estimators.classification.scikitlearn import ScikitlearnClassifier


class DeepForestCascadeForestClassifier(ScikitlearnClassifier):
    """ Wrapper class for deepforest.CascadeForestClassifier to be compatible with ART. """
    def __init__(
        self,
        model: "deepforest.CascadeForestClassifier",
        clip_values: Optional["CLIP_VALUES_TYPE"] = None,
        preprocessing_defences: Union["Preprocessor", List["Preprocessor"], None] = None,
        postprocessing_defences: Union["Postprocessor", List["Postprocessor"], None] = None,
        preprocessing: "PREPROCESSING_TYPE" = (0.0, 1.0),
    ) -> None:
        import deepforest

        if not isinstance(model, deepforest.CascadeForestClassifier):
            raise TypeError(f"Model must be of type deepforest.CascadeForestClassifier.")

        super().__init__(
            model=model,
            clip_values=clip_values,
            preprocessing_defences=preprocessing_defences,
            postprocessing_defences=postprocessing_defences,
            preprocessing=preprocessing,
        )

df = models.DF(random_state=seed)
df.fit(dataset.X_train, dataset.y_train)

/opt/homebrew/Caskroom/miniconda/base/envs/apollon/lib/python3.11/site-packages/deepforest/_estimator.py:212: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, sample_weight)
/opt/homebrew/Caskroom/miniconda/base/envs/apollon/lib/python3.11/site-packages/deepforest/_estimator.py:212: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, sample_weight)
/opt/homebrew/Caskroom/miniconda/base/envs/apollon/lib/python3.11/site-packages/deepforest/_estimator.py:212: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, sample_weight)
/opt/homebrew/Caskroom/miniconda/base/envs/apollon/lib/python3.11/site-packa

In [12]:
ifo = models.IF(random_state=seed)
ifo.fit(dataset.X_train, dataset.y_train)

IsolationForest(random_state=42, verbose=1)

# Generate adv data and evaluate models

In [13]:
mlp_testdata = evaluate.generate_test_data(dataset=dataset, attack_type=attack, model=mlp, n=n)
mlp_traindata = evaluate.generate_train_data(dataset=dataset, attack_type=attack, model=mlp, n=n)
evaluate.evaluate_model_on_testdata_using_metric(model=mlp, test_data=mlp_testdata, metric_type=metrics.BasicMetrics)

INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:01<00:00, 441.42it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 95.40%
INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:01<00:00, 445.51it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 95.40%


Metrics for MLPClassifier
--------------------------------------------------
Test accuracy: 0.967
Test precision: 1.0
Test recall: 0.934
Test f1: 0.9658738366080661
Test auc: 0.9670000000000001
Test confusion matrix:
[[500   0]
 [ 33 467]]
Test classification report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       500
           1       1.00      0.93      0.97       500

    accuracy                           0.97      1000
   macro avg       0.97      0.97      0.97      1000
weighted avg       0.97      0.97      0.97      1000

--------------------------------------------------


Metrics for MLPClassifier
--------------------------------------------------
Test accuracy: 0.523
Test precision: 1.0
Test recall: 0.046
Test f1: 0.08795411089866156
Test auc: 0.523
Test confusion matrix:
[[500   0]
 [477  23]]
Test classification report:
              precision    recall  f1-score   support

         0.0       0.51      1.00      0.

In [14]:
rf_testdata = evaluate.generate_test_data(dataset=dataset, attack_type=attack, model=rf, n=n)
rf_traindata = evaluate.generate_train_data(dataset=dataset, attack_type=attack, model=rf, n=n)
evaluate.evaluate_model_on_testdata_using_metric(model=rf, test_data=rf_testdata, metric_type=metrics.BasicMetrics)

INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:03<00:00, 133.65it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 70.20%
INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:04<00:00, 122.92it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 71.20%


Metrics for RandomForestClassifier
--------------------------------------------------
Test accuracy: 0.996
Test precision: 1.0
Test recall: 0.992
Test f1: 0.9959839357429718
Test auc: 0.996
Test confusion matrix:
[[500   0]
 [  4 496]]
Test classification report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       500
           1       1.00      0.99      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

--------------------------------------------------


Metrics for RandomForestClassifier
--------------------------------------------------
Test accuracy: 0.649
Test precision: 1.0
Test recall: 0.298
Test f1: 0.4591679506933744
Test auc: 0.649
Test confusion matrix:
[[500   0]
 [351 149]]
Test classification report:
              precision    recall  f1-score   support

         0.0       0.59      1.00    

In [15]:
dt_testdata = evaluate.generate_test_data(dataset=dataset, attack_type=attack, model=dt, n=n)
dt_traindata = evaluate.generate_train_data(dataset=dataset, attack_type=attack, model=dt, n=n)
evaluate.evaluate_model_on_testdata_using_metric(model=dt, test_data=dt_testdata, metric_type=metrics.BasicMetrics)

INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:00<00:00, 519.99it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 46.80%
INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:00<00:00, 517.92it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 46.60%


Metrics for DecisionTreeClassifier
--------------------------------------------------
Test accuracy: 0.997
Test precision: 1.0
Test recall: 0.994
Test f1: 0.9969909729187563
Test auc: 0.997
Test confusion matrix:
[[500   0]
 [  3 497]]
Test classification report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       500
           1       1.00      0.99      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

--------------------------------------------------


Metrics for DecisionTreeClassifier
--------------------------------------------------
Test accuracy: 0.766
Test precision: 1.0
Test recall: 0.532
Test f1: 0.6945169712793734
Test auc: 0.766
Test confusion matrix:
[[500   0]
 [234 266]]
Test classification report:
              precision    recall  f1-score   support

         0.0       0.68      1.00    

In [16]:
nb_testdata = evaluate.generate_test_data(dataset=dataset, attack_type=attack, model=nb, n=n)
nb_traindata = evaluate.generate_train_data(dataset=dataset, attack_type=attack, model=nb, n=n)
evaluate.evaluate_model_on_testdata_using_metric(model=nb, test_data=nb_testdata, metric_type=metrics.BasicMetrics)

INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:01<00:00, 417.98it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 48.60%
INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:01<00:00, 415.58it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 48.80%


Metrics for GaussianNB
--------------------------------------------------
Test accuracy: 0.834
Test precision: 0.7537993920972644
Test recall: 0.992
Test f1: 0.8566493955094991
Test auc: 0.8339999999999999
Test confusion matrix:
[[338 162]
 [  4 496]]
Test classification report:
              precision    recall  f1-score   support

           0       0.99      0.68      0.80       500
           1       0.75      0.99      0.86       500

    accuracy                           0.83      1000
   macro avg       0.87      0.83      0.83      1000
weighted avg       0.87      0.83      0.83      1000

--------------------------------------------------


Metrics for GaussianNB
--------------------------------------------------
Test accuracy: 0.595
Test precision: 0.6133651551312649
Test recall: 0.514
Test f1: 0.55930359085963
Test auc: 0.595
Test confusion matrix:
[[338 162]
 [243 257]]
Test classification report:
              precision    recall  f1-score   support

         0.0       0

In [17]:
lr_testdata = evaluate.generate_test_data(dataset=dataset, attack_type=attack, model=dt, n=n)
lr_traindata = evaluate.generate_train_data(dataset=dataset, attack_type=attack, model=dt, n=n)
evaluate.evaluate_model_on_testdata_using_metric(model=lr, test_data=lr_testdata, metric_type=metrics.BasicMetrics)

INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:00<00:00, 509.49it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 46.80%
INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:01<00:00, 487.43it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 46.80%


Metrics for LogisticRegression
--------------------------------------------------
Test accuracy: 0.789
Test precision: 0.9965635738831615
Test recall: 0.58
Test f1: 0.7332490518331226
Test auc: 0.789
Test confusion matrix:
[[499   1]
 [210 290]]
Test classification report:
              precision    recall  f1-score   support

           0       0.70      1.00      0.83       500
           1       1.00      0.58      0.73       500

    accuracy                           0.79      1000
   macro avg       0.85      0.79      0.78      1000
weighted avg       0.85      0.79      0.78      1000

--------------------------------------------------


Metrics for LogisticRegression
--------------------------------------------------
Test accuracy: 0.787
Test precision: 0.9965397923875432
Test recall: 0.576
Test f1: 0.7300380228136882
Test auc: 0.787
Test confusion matrix:
[[499   1]
 [212 288]]
Test classification report:
              precision    recall  f1-score   support

         0.0    

In [18]:
gbc_testdata = evaluate.generate_test_data(dataset=dataset, attack_type=attack, model=gbc, n=n)
gbc_traindata = evaluate.generate_train_data(dataset=dataset, attack_type=attack, model=gbc, n=n)
evaluate.evaluate_model_on_testdata_using_metric(model=gbc, test_data=gbc_testdata, metric_type=metrics.BasicMetrics)

INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:01<00:00, 359.64it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 44.40%
INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:01<00:00, 344.61it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 44.00%


Metrics for GradientBoostingClassifier
--------------------------------------------------
Test accuracy: 0.974
Test precision: 0.9978991596638656
Test recall: 0.95
Test f1: 0.9733606557377049
Test auc: 0.974
Test confusion matrix:
[[499   1]
 [ 25 475]]
Test classification report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       500
           1       1.00      0.95      0.97       500

    accuracy                           0.97      1000
   macro avg       0.98      0.97      0.97      1000
weighted avg       0.98      0.97      0.97      1000

--------------------------------------------------


Metrics for GradientBoostingClassifier
--------------------------------------------------
Test accuracy: 0.777
Test precision: 0.996415770609319
Test recall: 0.556
Test f1: 0.7137355584082157
Test auc: 0.777
Test confusion matrix:
[[499   1]
 [222 278]]
Test classification report:
              precision    recall  f1-score   support

 

In [19]:
et_testdata = evaluate.generate_test_data(dataset=dataset, attack_type=attack, model=et, n=n)
et_traindata = evaluate.generate_train_data(dataset=dataset, attack_type=attack, model=et, n=n)
evaluate.evaluate_model_on_testdata_using_metric(model=et, test_data=et_testdata, metric_type=metrics.BasicMetrics)

INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:04<00:00, 108.38it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 64.60%
INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:04<00:00, 118.33it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 63.40%


Metrics for ExtraTreesClassifier
--------------------------------------------------
Test accuracy: 0.996
Test precision: 1.0
Test recall: 0.992
Test f1: 0.9959839357429718
Test auc: 0.996
Test confusion matrix:
[[500   0]
 [  4 496]]
Test classification report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       500
           1       1.00      0.99      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

--------------------------------------------------


Metrics for ExtraTreesClassifier
--------------------------------------------------
Test accuracy: 0.677
Test precision: 1.0
Test recall: 0.354
Test f1: 0.5228951255539144
Test auc: 0.677
Test confusion matrix:
[[500   0]
 [323 177]]
Test classification report:
              precision    recall  f1-score   support

         0.0       0.61      1.00      0.

In [20]:
df_testdata = evaluate.generate_test_data(dataset=dataset, attack_type=attack, model=DeepForestCascadeForestClassifier(df), n=n)
df_traindata = evaluate.generate_train_data(dataset=dataset, attack_type=attack, model=DeepForestCascadeForestClassifier(df), n=n)
evaluate.evaluate_model_on_testdata_using_metric(model=df, test_data=df_testdata, metric_type=metrics.BasicMetrics)

INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:15<00:00, 31.58it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 92.80%
INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:15<00:00, 33.14it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 93.20%


Metrics for CascadeForestClassifier
--------------------------------------------------
Test accuracy: 0.997
Test precision: 0.9979959919839679
Test recall: 0.996
Test f1: 0.996996996996997
Test auc: 0.997
Test confusion matrix:
[[499   1]
 [  2 498]]
Test classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

--------------------------------------------------


Metrics for CascadeForestClassifier
--------------------------------------------------
Test accuracy: 0.535
Test precision: 0.972972972972973
Test recall: 0.072
Test f1: 0.1340782122905028
Test auc: 0.535
Test confusion matrix:
[[499   1]
 [464  36]]
Test classification report:
              precision    recall  f1-score   support

       

In [21]:
ifo_testdata = evaluate.generate_test_data(dataset=dataset, attack_type=attack, model=ifo, n=n)
ifo_traindata = evaluate.generate_train_data(dataset=dataset, attack_type=attack, model=ifo, n=n)
evaluate.evaluate_model_on_testdata_using_metric(model=ifo, test_data=ifo_testdata, metric_type=metrics.BasicMetrics)

INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:04<00:00, 107.29it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 71.20%
INFO:art.attacks.evasion.zoo:Disable resizing and importance sampling because feature vector input has been detected.
ZOO: 100%|██████████| 500/500 [00:04<00:00, 100.53it/s]
INFO:art.attacks.evasion.zoo:Success rate of ZOO attack: 71.20%


Metrics for IsolationForest
--------------------------------------------------
Test accuracy: 0.621
Test precision: 0.8579881656804734
Test recall: 0.29
Test f1: 0.4334828101644245
Test auc: 0.621
Test confusion matrix:
[[476  24]
 [355 145]]
Test classification report:
              precision    recall  f1-score   support

           0       0.57      0.95      0.72       500
           1       0.86      0.29      0.43       500

    accuracy                           0.62      1000
   macro avg       0.72      0.62      0.57      1000
weighted avg       0.72      0.62      0.57      1000

--------------------------------------------------


Metrics for IsolationForest
--------------------------------------------------
Test accuracy: 0.62
Test precision: 0.8571428571428571
Test recall: 0.288
Test f1: 0.4311377245508982
Test auc: 0.62
Test confusion matrix:
[[476  24]
 [356 144]]
Test classification report:
              precision    recall  f1-score   support

         0.0       0.57 

# Fit MABs

In [22]:
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score

mab3 = apollon.MABTrainer(arms=[
    models.RF(random_state=seed),
    models.DT(random_state=seed),
    models.NB(),
], n_clusters=2, random_state=seed, scorer=make_scorer(accuracy_score))
mab3.fit(dataset.X_train, dataset.y_train)

INFO:root:Cluster 0: 603379 samples: (array([0, 1], dtype=int8), array([520716,  82663]))
INFO:root:Training RandomForestClassifier on cluster 0
INFO:root:Setting reward for RandomForestClassifier on cluster 0
INFO:root:Time to fit: 202.95596289634705 s
INFO:root:Time to cross validate: 209.95700883865356 s
INFO:root:Setting reward for RandomForestClassifier on cluster 0: 0.9983062055139382
INFO:root:Training DecisionTreeClassifier on cluster 0
INFO:root:Setting reward for DecisionTreeClassifier on cluster 0
INFO:root:Time to fit: 55.67437291145325 s
INFO:root:Time to cross validate: 45.11649799346924 s
INFO:root:Setting reward for DecisionTreeClassifier on cluster 0: 0.9979217041863846
INFO:root:Training GaussianNB on cluster 0
INFO:root:Setting reward for GaussianNB on cluster 0
INFO:root:Time to fit: 0.24998688697814941 s
INFO:root:Time to cross validate: 0.7123830318450928 s
INFO:root:Setting reward for GaussianNB on cluster 0: 0.5203943289990778
INFO:root:Cluster 1: 893249 samples

In [23]:
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score

mab5 = apollon.MABTrainer(arms=[
    models.RF(random_state=seed),
    models.DT(random_state=seed),
    models.NB(),
    models.LR(random_state=seed),
    models.MLP(random_state=seed),
], n_clusters=2, random_state=seed, scorer=make_scorer(accuracy_score))
mab5.fit(dataset.X_train, dataset.y_train)

INFO:root:Cluster 0: 603379 samples: (array([0, 1], dtype=int8), array([520716,  82663]))
INFO:root:Training RandomForestClassifier on cluster 0
INFO:root:Setting reward for RandomForestClassifier on cluster 0
INFO:root:Time to fit: 192.99544382095337 s
INFO:root:Time to cross validate: 173.6222152709961 s
INFO:root:Setting reward for RandomForestClassifier on cluster 0: 0.9983062055139382
INFO:root:Training DecisionTreeClassifier on cluster 0
INFO:root:Setting reward for DecisionTreeClassifier on cluster 0
INFO:root:Time to fit: 55.69451284408569 s
INFO:root:Time to cross validate: 45.107009172439575 s
INFO:root:Setting reward for DecisionTreeClassifier on cluster 0: 0.9979217041863846
INFO:root:Training GaussianNB on cluster 0
INFO:root:Setting reward for GaussianNB on cluster 0
INFO:root:Time to fit: 0.32596492767333984 s
INFO:root:Time to cross validate: 0.761599063873291 s
INFO:root:Setting reward for GaussianNB on cluster 0: 0.5203943289990778
INFO:root:Training LogisticRegressio

In [24]:
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score

mab9 = apollon.MABTrainer(arms=[
    models.RF(random_state=seed),
    models.DT(random_state=seed),
    models.NB(),
    models.LR(random_state=seed),
    models.MLP(random_state=seed),
    models.GBC(random_state=seed),
    models.ET(random_state=seed),
    models.DF(random_state=seed),
    models.IF(random_state=seed),
], n_clusters=2, random_state=seed, scorer=make_scorer(accuracy_score))
mab9.fit(dataset.X_train, dataset.y_train)

INFO:root:Cluster 0: 603379 samples: (array([0, 1], dtype=int8), array([520716,  82663]))
INFO:root:Training RandomForestClassifier on cluster 0
INFO:root:Setting reward for RandomForestClassifier on cluster 0
INFO:root:Time to fit: 190.96462631225586 s
INFO:root:Time to cross validate: 171.5290880203247 s
INFO:root:Setting reward for RandomForestClassifier on cluster 0: 0.9983062055139382
INFO:root:Training DecisionTreeClassifier on cluster 0
INFO:root:Setting reward for DecisionTreeClassifier on cluster 0
INFO:root:Time to fit: 55.23582696914673 s
INFO:root:Time to cross validate: 44.72688317298889 s
INFO:root:Setting reward for DecisionTreeClassifier on cluster 0: 0.9979217041863846
INFO:root:Training GaussianNB on cluster 0
INFO:root:Setting reward for GaussianNB on cluster 0
INFO:root:Time to fit: 0.3318469524383545 s
INFO:root:Time to cross validate: 0.7225220203399658 s
INFO:root:Setting reward for GaussianNB on cluster 0: 0.5203943289990778
INFO:root:Training LogisticRegression

In [25]:
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score

# Evaluate best models on normal dataset
accuracy_of = lambda model: metrics.BasicMetrics(model=model, X_train=dataset.X_train, y_train=dataset.y_train, X_test=dataset.X_test, y_test=dataset.y_test).train_accuracy()

models_accuracy = {
    rf: [None, rf_testdata],
    dt: [None, dt_testdata],
    nb: [None, nb_testdata],
    lr: [None, lr_testdata],
    mlp: [None, mlp_testdata],
    gbc: [None, gbc_testdata],
    et: [None, et_testdata],
    df: [None, df_testdata],
    ifo: [None, ifo_testdata],
}

for model in models_accuracy:
    models_accuracy[model][0] = accuracy_of(model)

models_accuracy = dict(sorted(models_accuracy.items(), key=lambda item: item[1][0], reverse=True))

mab_top4 = apollon.MABTrainer(arms=list(models_accuracy.keys())[:4], n_clusters=2, random_state=seed, scorer=make_scorer(accuracy_score))
mab_top4.fit(dataset.X_train, dataset.y_train)

INFO:root:Cluster 0: 603379 samples: (array([0, 1], dtype=int8), array([520716,  82663]))
INFO:root:Training ExtraTreesClassifier on cluster 0
INFO:root:Setting reward for ExtraTreesClassifier on cluster 0
INFO:root:Time to fit: 54.17340803146362 s
INFO:root:Time to cross validate: 47.755332946777344 s
INFO:root:Setting reward for ExtraTreesClassifier on cluster 0: 0.9992425987728621
INFO:root:Training DecisionTreeClassifier on cluster 0
INFO:root:Setting reward for DecisionTreeClassifier on cluster 0
INFO:root:Time to fit: 55.937971115112305 s
INFO:root:Time to cross validate: 45.253369092941284 s
INFO:root:Setting reward for DecisionTreeClassifier on cluster 0: 0.9979217041863846
INFO:root:Training CascadeForestClassifier on cluster 0
/opt/homebrew/Caskroom/miniconda/base/envs/apollon/lib/python3.11/site-packages/deepforest/_estimator.py:212: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usi

In [26]:
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score

# Evaluate best models on adv dataset
models_accuracy_adv = {
    rf: [metrics.BasicMetrics(model=rf, X_train=rf_traindata[1][0], y_train=rf_traindata[1][1], X_test=rf_testdata[1][0], y_test=rf_testdata[1][1]).train_accuracy(), rf_traindata],
    dt: [metrics.BasicMetrics(model=dt, X_train=dt_traindata[1][0], y_train=dt_traindata[1][1], X_test=dt_testdata[1][0], y_test=dt_testdata[1][1]).train_accuracy(), dt_traindata],
    nb: [metrics.BasicMetrics(model=nb, X_train=nb_traindata[1][0], y_train=nb_traindata[1][1], X_test=nb_testdata[1][0], y_test=nb_testdata[1][1]).train_accuracy(), nb_traindata],
    lr: [metrics.BasicMetrics(model=lr, X_train=lr_traindata[1][0], y_train=lr_traindata[1][1], X_test=lr_testdata[1][0], y_test=lr_testdata[1][1]).train_accuracy(), lr_traindata],
    mlp: [metrics.BasicMetrics(model=mlp, X_train=mlp_traindata[1][0], y_train=mlp_traindata[1][1], X_test=mlp_testdata[1][0], y_test=mlp_testdata[1][1]).train_accuracy(), mlp_traindata],
    gbc: [metrics.BasicMetrics(model=gbc, X_train=gbc_traindata[1][0], y_train=gbc_traindata[1][1], X_test=gbc_testdata[1][0], y_test=gbc_testdata[1][1]).train_accuracy(), gbc_traindata],
    et: [metrics.BasicMetrics(model=et, X_train=et_traindata[1][0], y_train=et_traindata[1][1], X_test=et_testdata[1][0], y_test=et_testdata[1][1]).train_accuracy(), et_traindata],
    df: [metrics.BasicMetrics(model=df, X_train=df_traindata[1][0], y_train=df_traindata[1][1], X_test=df_testdata[1][0], y_test=df_testdata[1][1]).train_accuracy(), df_traindata],
    ifo: [metrics.BasicMetrics(model=ifo, X_train=ifo_traindata[1][0], y_train=ifo_traindata[1][1], X_test=ifo_testdata[1][0], y_test=ifo_testdata[1][1]).train_accuracy(), ifo_traindata],
}

models_accuracy_adv = dict(sorted(models_accuracy_adv.items(), key=lambda item: item[1][0], reverse=True))

mab_top4adv = apollon.MABTrainer(arms=list(models_accuracy_adv.keys())[:4], n_clusters=2, random_state=seed, scorer=make_scorer(accuracy_score))
mab_top4adv.fit(dataset.X_train, dataset.y_train)

INFO:root:Cluster 0: 603379 samples: (array([0, 1], dtype=int8), array([520716,  82663]))
INFO:root:Training LogisticRegression on cluster 0
INFO:root:Setting reward for LogisticRegression on cluster 0
/opt/homebrew/Caskroom/miniconda/base/envs/apollon/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
INFO:root:Time to fit: 5.330678939819336 s
INFO:root:Time to cross validate: 7.050357103347778 s
INFO:root:Setting reward for LogisticRegression on cluster 0: 0.8852561989772207
INFO:root:Training GradientBoostingClassifier on cluster 0
INFO:root:

In [27]:
mab_top4.arms[0]

[ExtraTreesClassifier(random_state=42),
 DecisionTreeClassifier(random_state=42),
 CascadeForestClassifier(random_state=42, verbose=0),
 RandomForestClassifier(random_state=42)]

In [28]:
mab_top4adv.arms[0]

[LogisticRegression(random_state=42),
 GradientBoostingClassifier(random_state=42),
 DecisionTreeClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42)]

# Evaluate MABs

## Standard Thompson Sampling

In [29]:
thompson_predictor = lambda mab: MABThompsonSampling(mab=mab)

### non-adv

In [30]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=thompson_predictor(mab3),
    testdatas=[rf_testdata, dt_testdata, nb_testdata],
    metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.9606666666666666
INFO:root:Average precision: 0.9308291158278915
INFO:root:Average recall: 0.9953333333333333
INFO:root:Average f1: 0.9619920268460693
INFO:root:Average auc: 0.9606666666666667


In [31]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=thompson_predictor(mab5),
    testdatas=[rf_testdata, dt_testdata, nb_testdata, lr_testdata, mlp_testdata],
    metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.9406000000000001
INFO:root:Average precision: 0.9581106177259576
INFO:root:Average recall: 0.9216000000000001
INFO:root:Average f1: 0.9394682989756962
INFO:root:Average auc: 0.9406000000000001


In [32]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=thompson_predictor(mab9),
    testdatas=[rf_testdata, dt_testdata, nb_testdata, lr_testdata, mlp_testdata, gbc_testdata, et_testdata, df_testdata, ifo_testdata], metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.9676666666666667
INFO:root:Average precision: 0.9834853529768177
INFO:root:Average recall: 0.9513333333333333
INFO:root:Average f1: 0.9671184339939773
INFO:root:Average auc: 0.9676666666666667


In [33]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=thompson_predictor(mab_top4),
    testdatas=[et_testdata, dt_testdata, df_testdata, rf_testdata],
    metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.9965
INFO:root:Average precision: 1.0
INFO:root:Average recall: 0.993
INFO:root:Average f1: 0.9964864437543889
INFO:root:Average auc: 0.9965


In [34]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=thompson_predictor(mab_top4adv),
    testdatas=[lr_testdata, gbc_testdata, dt_testdata, et_testdata],
    metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.96375
INFO:root:Average precision: 0.9978609601200608
INFO:root:Average recall: 0.9295000000000001
INFO:root:Average f1: 0.9624640550221986
INFO:root:Average auc: 0.9637500000000001


### adv

In [35]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=thompson_predictor(mab3),
    testdatas=[rf_testdata, dt_testdata, nb_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.7453333333333334
INFO:root:Average precision: 0.8798092879500462
INFO:root:Average recall: 0.5640000000000001
INFO:root:Average f1: 0.681274710960425
INFO:root:Average auc: 0.7453333333333333


In [36]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=thompson_predictor(mab5),
    testdatas=[rf_testdata, dt_testdata, nb_testdata, lr_testdata, mlp_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.7598
INFO:root:Average precision: 0.9279713426727045
INFO:root:Average recall: 0.5604
INFO:root:Average f1: 0.6912753897278954
INFO:root:Average auc: 0.7597999999999999


In [37]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=thompson_predictor(mab9),
    testdatas=[rf_testdata, dt_testdata, nb_testdata, lr_testdata, mlp_testdata, gbc_testdata, et_testdata, df_testdata, ifo_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.7924444444444444
INFO:root:Average precision: 0.9618388732792411
INFO:root:Average recall: 0.6082222222222222
INFO:root:Average f1: 0.7320322245535082
INFO:root:Average auc: 0.7924444444444444


In [38]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=thompson_predictor(mab_top4),
    testdatas=[et_testdata, dt_testdata, df_testdata, rf_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.7404999999999999
INFO:root:Average precision: 1.0
INFO:root:Average recall: 0.481
INFO:root:Average f1: 0.6353668730299614
INFO:root:Average auc: 0.7404999999999999


In [39]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=thompson_predictor(mab_top4adv),
    testdatas=[lr_testdata, gbc_testdata, dt_testdata, et_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.8125
INFO:root:Average precision: 0.996810887558334
INFO:root:Average recall: 0.627
INFO:root:Average f1: 0.7688995460734591
INFO:root:Average auc: 0.8125


## Epsilon Greedy

In [40]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=MABEpsilonGreedy(mab=mab_top4adv, epsilon=0.10),
    testdatas=[lr_traindata, gbc_traindata, dt_traindata, et_traindata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.83375
INFO:root:Average precision: 1.0
INFO:root:Average recall: 0.6675
INFO:root:Average f1: 0.7994621972695057
INFO:root:Average auc: 0.83375


In [41]:
epsilon_predictor = lambda mab: MABEpsilonGreedy(mab=mab, epsilon=0.07)

### non-adv

In [42]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=epsilon_predictor(mab3),
    testdatas=[rf_testdata, dt_testdata, nb_testdata],
    metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.9916666666666667
INFO:root:Average precision: 0.9893837506171471
INFO:root:Average recall: 0.9940000000000001
INFO:root:Average f1: 0.9916862851233392
INFO:root:Average auc: 0.9916666666666667


In [43]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=epsilon_predictor(mab5),
    testdatas=[rf_testdata, dt_testdata, nb_testdata, lr_testdata, mlp_testdata],
    metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.9916
INFO:root:Average precision: 0.9951653370860548
INFO:root:Average recall: 0.9880000000000001
INFO:root:Average f1: 0.9915690803111034
INFO:root:Average auc: 0.9916


In [44]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=epsilon_predictor(mab9),
    testdatas=[rf_testdata, dt_testdata, nb_testdata, lr_testdata, mlp_testdata, gbc_testdata, et_testdata, df_testdata, ifo_testdata], metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.9922222222222222
INFO:root:Average precision: 0.9961935009817918
INFO:root:Average recall: 0.988222222222222
INFO:root:Average f1: 0.9921882676561368
INFO:root:Average auc: 0.9922222222222222


In [45]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=epsilon_predictor(mab_top4),
    testdatas=[et_testdata, dt_testdata, df_testdata, rf_testdata],
    metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.998
INFO:root:Average precision: 1.0
INFO:root:Average recall: 0.996
INFO:root:Average f1: 0.9979959919839679
INFO:root:Average auc: 0.998


In [46]:
evaluate.evaluate_apollon_on_non_adv_testdata_using_metric(
    apollon=epsilon_predictor(mab_top4adv),
    testdatas=[lr_testdata, gbc_testdata, dt_testdata, et_testdata],
    metric_type=metrics.BasicMetrics
)

INFO:root:Average accuracy: 0.993
INFO:root:Average precision: 1.0
INFO:root:Average recall: 0.986
INFO:root:Average f1: 0.9929491210884609
INFO:root:Average auc: 0.993


### adv

In [47]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=epsilon_predictor(mab3),
    testdatas=[rf_testdata, dt_testdata, nb_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.7736666666666666
INFO:root:Average precision: 0.9869168892468059
INFO:root:Average recall: 0.5539999999999999
INFO:root:Average f1: 0.6934684688987377
INFO:root:Average auc: 0.7736666666666667


In [48]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=epsilon_predictor(mab5),
    testdatas=[rf_testdata, dt_testdata, nb_testdata, lr_testdata, mlp_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.7634000000000001
INFO:root:Average precision: 0.9915672410836072
INFO:root:Average recall: 0.5312
INFO:root:Average f1: 0.6742526043859768
INFO:root:Average auc: 0.7634000000000001


In [49]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=epsilon_predictor(mab9),
    testdatas=[rf_testdata, dt_testdata, nb_testdata, lr_testdata, mlp_testdata, gbc_testdata, et_testdata, df_testdata, ifo_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.8161111111111111
INFO:root:Average precision: 0.9919446369750352
INFO:root:Average recall: 0.634888888888889
INFO:root:Average f1: 0.7271351507012834
INFO:root:Average auc: 0.8161111111111111


In [50]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=epsilon_predictor(mab_top4),
    testdatas=[et_testdata, dt_testdata, df_testdata, rf_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.76675
INFO:root:Average precision: 1.0
INFO:root:Average recall: 0.5335
INFO:root:Average f1: 0.6479656504648694
INFO:root:Average auc: 0.76675


In [51]:
evaluate.evaluate_apollon_on_testdata_using_metric(
    apollon=epsilon_predictor(mab_top4adv),
    testdatas=[lr_testdata, gbc_testdata, dt_testdata, et_testdata],
    metric_type=metrics.BasicMetrics,
)

INFO:root:Average accuracy: 0.839
INFO:root:Average precision: 1.0
INFO:root:Average recall: 0.678
INFO:root:Average f1: 0.806409977512112
INFO:root:Average auc: 0.839
